# Carga de dados - textos de queries e enunciados

In [1]:
import pandas as pd

In [2]:
df_queries = pd.concat([pd.read_csv('../../data/juris_tcu/query1.csv', sep=';', index_col='QUERY_ID'),
                        pd.read_csv('../../data/juris_tcu/query2.csv', sep=';', index_col='QUERY_ID'),
                        pd.read_csv('../../data/juris_tcu/query3.csv', sep=';', index_col='QUERY_ID')])
df_queries

,QUERY_TEXT
QUERY_ID,
1,técnica e preço
2,restos a pagar
3,aditivo a contrato
4,adesão a ata de registro de preços
5,sobrepreço e superfaturamento
...,...
146,É possível a inclusão de serviços já previstos...
147,O princípio da legalidade estrita pode ser afa...
148,Quais fontes devem ser prioritárias na pesquis...


In [3]:
df_enunciados = pd.read_csv('../../data/juris_tcu/doc.csv', index_col='ID')[['TEXT']].rename_axis('DOC_ID')
df_enunciados.columns = ['DOC_TEXT']
df_enunciados

,DOC_TEXT
DOC_ID,
13568,É ilegal a contagem de tempo ficto de serviço ...
11614,"SÚMULA TCU 283: Para fim de habilitação, a Adm..."
21087,A contratação de serviços por preços superiore...
35016,"Não se aplica, excepcionalmente, multa aos ges..."
29370,"Em contratatações de TI, não cabe aceitar prop..."
...,...
55,"SÚMULA TCU 51: Quando, no exame e julgamento d..."
95,"SÚMULA TCU 91: A falta de remessa, em tempo há..."
94,"SÚMULA TCU 90: O Parecer Prévio, em sentido fa..."


# Carga de resultados de busca e criação de dados de avaliação

In [4]:
def build_eval_data(run_path, df_queries, df_docs):
    df_run = pd.read_csv(run_path)
    df_run = df_run.merge(df_queries, on='QUERY_ID')
    df_run = df_run.merge(df_docs, on='DOC_ID')
    return df_run

In [8]:
def write_results_query(df_run, result_out, query_id, K = 10):
    df_target = df_run.query('QUERY_ID == ' + str(query_id) + ' and RANK <= ' + str(K)).sort_values('RANK')
    query_text = df_target.iloc[0]['QUERY_TEXT']
    result_out.write('Query ' + str(query_id) + ' - ' + query_text + ('\n' if query_text[-1] == '?' else ':\n'))
    for rank, doc_id, doc_text in df_target[['RANK', 'DOC_ID', 'DOC_TEXT']].values:
        result_out.write('\t' + (' ' if rank < 10 else '') + str(rank) + ' - (' + str(doc_id) + ') ' + doc_text + '\n')

In [9]:
def write_results(df_run, result_path, K = 10):
    with open(result_path, 'w', encoding='Utf-8') as f_out:
        query_id = 1
        while True:
            write_results_query(df_run, f_out, query_id, K)
            if query_id <= 50:
                query_id += 1
            elif query_id <= 100:
                query_id += 50
            elif query_id < 150:
                query_id -= 49
            else:
                break

In [12]:
df = build_eval_data('../../data/search/juris_tcu/run_bm25.csv', df_queries, df_enunciados)
write_results(df, '../../data/search/juris_tcu/results_bm25.txt')
df

,QUERY_ID,RANK,DOC_ID,ENGINE,QUERY_TEXT,DOC_TEXT
0,1,1,20870,BM25,técnica e preço,"Nas licitações do tipo técnica e preço, a atri..."
1,5,281,20870,BM25,sobrepreço e superfaturamento,"Nas licitações do tipo técnica e preço, a atri..."
2,7,149,20870,BM25,acréscimos e supressões,"Nas licitações do tipo técnica e preço, a atri..."
3,10,124,20870,BM25,diarias e passagens,"Nas licitações do tipo técnica e preço, a atri..."
4,16,800,20870,BM25,inexequibilidade e comprovação,"Nas licitações do tipo técnica e preço, a atri..."
...,...,...,...,...,...,...
142078,149,920,85627,BM25,Devem ser aceitas nas contratações de software...,"A relação de dependência econômica, para fins ..."
142079,149,921,6969,BM25,Devem ser aceitas nas contratações de software...,"A relação de dependência econômica, para fins ..."
142080,149,982,30266,BM25,Devem ser aceitas nas contratações de software...,"O tempo de serviço exercido até 16/12/1998, da..."
142081,150,102,15329,BM25,Qual é a regra geral para a contratação de ent...,Os Serviços Sociais Autônomos observarão a reg...


In [13]:
df = build_eval_data('../../data/search/juris_tcu/run_sts.csv', df_queries, df_enunciados)
write_results(df, '../../data/search/juris_tcu/results_sts.txt')
df

,QUERY_ID,RANK,DOC_ID,ENGINE,QUERY_TEXT,DOC_TEXT
0,1,1,34233,SentenceTransformerS,técnica e preço,Os fatores de avaliação das propostas técnicas...
1,8,236,34233,SentenceTransformerS,obras e serviços de engenharia,Os fatores de avaliação das propostas técnicas...
2,11,240,34233,SentenceTransformerS,bens e serviços comuns,Os fatores de avaliação das propostas técnicas...
3,12,109,34233,SentenceTransformerS,parcelas de maior relevância e valor significa...,Os fatores de avaliação das propostas técnicas...
4,15,125,34233,SentenceTransformerS,contas e materialidade,Os fatores de avaliação das propostas técnicas...
...,...,...,...,...,...,...
74995,150,455,30044,SentenceTransformerS,Qual é a regra geral para a contratação de ent...,É ilegal a inclusão nos proventos de servidor ...
74996,150,461,19599,SentenceTransformerS,Qual é a regra geral para a contratação de ent...,"A partir de 18/5/2001, data da publicação da d..."
74997,150,462,82304,SentenceTransformerS,Qual é a regra geral para a contratação de ent...,"A transformação, por lei, de emprego público e..."
74998,150,464,110843,SentenceTransformerS,Qual é a regra geral para a contratação de ent...,"A transformação, por lei, de emprego público e..."


In [14]:
df = build_eval_data('../../data/search/juris_tcu/run_bm25_reranker.csv', df_queries, df_enunciados)
write_results(df, '../../data/search/juris_tcu/results_bm25_reranker.txt')
df

,QUERY_ID,RANK,DOC_ID,ENGINE,QUERY_TEXT,DOC_TEXT
0,1,1,21064,BM25+Reranker,técnica e preço,Admite-se a preponderância da técnica sobre o ...
1,6,99,21064,BM25+Reranker,restrição a competitividade,Admite-se a preponderância da técnica sobre o ...
2,31,299,21064,BM25+Reranker,medida cautelar,Admite-se a preponderância da técnica sobre o ...
3,72,2,21064,BM25+Reranker,ponderação técnica e preço,Admite-se a preponderância da técnica sobre o ...
4,73,68,21064,BM25+Reranker,pontuação desarrazoada licitações técnica e preço,Admite-se a preponderância da técnica sobre o ...
...,...,...,...,...,...,...
44178,150,280,33564,BM25+Reranker,Qual é a regra geral para a contratação de ent...,"É reconhecido o direito à ""opção"" aos servidor..."
44179,150,285,130287,BM25+Reranker,Qual é a regra geral para a contratação de ent...,Em casos de acumulação de remuneração e pensão...
44180,150,286,141744,BM25+Reranker,Qual é a regra geral para a contratação de ent...,Em casos de acumulação de remuneração e pensão...
44181,150,289,13169,BM25+Reranker,Qual é a regra geral para a contratação de ent...,A regra geral que confere efeito suspensivo ao...


In [15]:
df = build_eval_data('../../data/search/juris_tcu/run_sts_reranker.csv', df_queries, df_enunciados)
write_results(df, '../../data/search/juris_tcu/results_sts_reranker.txt')
df

,QUERY_ID,RANK,DOC_ID,ENGINE,QUERY_TEXT,DOC_TEXT
0,1,1,21064,STS+Reranker,técnica e preço,Admite-se a preponderância da técnica sobre o ...
1,12,269,21064,STS+Reranker,parcelas de maior relevância e valor significa...,Admite-se a preponderância da técnica sobre o ...
2,72,2,21064,STS+Reranker,ponderação técnica e preço,Admite-se a preponderância da técnica sobre o ...
3,73,60,21064,STS+Reranker,pontuação desarrazoada licitações técnica e preço,Admite-se a preponderância da técnica sobre o ...
4,85,6,21064,STS+Reranker,técnica e preço profissionais quadro permanente,Admite-se a preponderância da técnica sobre o ...
...,...,...,...,...,...,...
44995,150,293,25628,STS+Reranker,Qual é a regra geral para a contratação de ent...,Atividades não relacionadas às específicas dos...
44996,150,296,28500,STS+Reranker,Qual é a regra geral para a contratação de ent...,As decisões judiciais sobre incorporações de p...
44997,150,297,30125,STS+Reranker,Qual é a regra geral para a contratação de ent...,As decisões judiciais sobre incorporações de p...
44998,150,299,18989,STS+Reranker,Qual é a regra geral para a contratação de ent...,"Ao assumir o cargo, compete ao gestor público ..."


In [16]:
df = build_eval_data('../../data/search/juris_tcu/run_join_bm25_sts_reranker.csv', df_queries, df_enunciados)
write_results(df, '../../data/search/juris_tcu/results_join_bm25_sts_reranker.txt')
df

,QUERY_ID,RANK,DOC_ID,ENGINE,QUERY_TEXT,DOC_TEXT
0,1,1,21064,(BM25|STS)+Reranker,técnica e preço,Admite-se a preponderância da técnica sobre o ...
1,6,121,21064,(BM25|STS)+Reranker,restrição a competitividade,Admite-se a preponderância da técnica sobre o ...
2,12,491,21064,(BM25|STS)+Reranker,parcelas de maior relevância e valor significa...,Admite-se a preponderância da técnica sobre o ...
3,31,526,21064,(BM25|STS)+Reranker,medida cautelar,Admite-se a preponderância da técnica sobre o ...
4,72,2,21064,(BM25|STS)+Reranker,ponderação técnica e preço,Admite-se a preponderância da técnica sobre o ...
...,...,...,...,...,...,...
82026,150,511,28527,(BM25|STS)+Reranker,Qual é a regra geral para a contratação de ent...,A aposentadoria fundamentada na Lei Complement...
82027,150,521,33564,(BM25|STS)+Reranker,Qual é a regra geral para a contratação de ent...,"É reconhecido o direito à ""opção"" aos servidor..."
82028,150,539,28500,(BM25|STS)+Reranker,Qual é a regra geral para a contratação de ent...,As decisões judiciais sobre incorporações de p...
82029,150,540,30125,(BM25|STS)+Reranker,Qual é a regra geral para a contratação de ent...,As decisões judiciais sobre incorporações de p...
